In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dataset=pd.read_csv('/kaggle/input/up-crime-data-year-2019/MR data - Compiled Data Set.csv')
dataset.head()

In [ ]:
new_columns=dataset.columns.values
new_columns[0]='City'
new_columns[3]="Cause of murder"
new_columns[10]="Crimes under IPC Act"
new_columns[5]="Adult Male Victims"
new_columns[6]="Adult Female Victims"
new_columns[11]="Adult Victims"
new_columns[12]="Number of Children"
new_columns[4]="Child Victims"
dataset.head()
dataset.columns=new_columns


In [ ]:
dataset=dataset.drop(['Title','Text'],axis=1)
dataset.head()

In [ ]:
dataset=dataset.fillna(0)
dataset.head(20)

In [ ]:
for i in range(1,12):
    dataset.insert(7+i,"Act{}".format(i),0)

In [ ]:
dataset.head()

In [ ]:
dataset['Crimes under IPC Act']=dataset['Crimes under IPC Act'].astype('string')
crime_act_values=(dataset['Crimes under IPC Act'].values)
for j in range(len(crime_act_values)):
    acts_committed=crime_act_values[j].split(',')
    for x in acts_committed:
        if x=='0':
            continue
        dataset.loc[int(j),'Act{}'.format(x.strip())]=1
        

In [ ]:
dataset.head(25)

In [ ]:
city_wise=(dataset['City'].value_counts().to_dict())

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
values=list(city_wise.values())
city_name=list(city_wise.keys())
plt.pie(values,labels=city_name,autopct='%.2f%%')
plt.title("City wise crime data")
plt.show()

In [ ]:
murder_causes=dataset["Cause of murder"].value_counts().to_dict()
murder_causes.pop(0)
m_value=list(murder_causes.values())
m_key=list(murder_causes.keys())
plt.bar(range(len(murder_causes)),m_value,tick_label=m_key)
plt.xticks(rotation=90)
plt.show()

In [ ]:
my_dict={}
for i in range(1,12):
    my_dict["Act {}".format(i)]=dataset["Act{}".format(i)].sum() 

act_val = list(my_dict.values())
act_key = list(my_dict.keys())
plt.bar(range(len(my_dict)),act_val,tick_label = act_key)
plt.ylabel("Number of crimes committed",fontsize=10)
plt.title("IPC Act wise crimes committed ",fontsize=10)
plt.xticks(rotation=90,fontsize=10)

plt.show()

In [ ]:
count = {}
for i in range(1,12):
    count[i]=[0,0]
for i in range(len(dataset['City'])):
    for actno in range(1,12):
        if dataset.loc[i,"Act{}".format(actno)]!=0:
            count[actno][0]+=int(dataset.loc[i,"Number of Children"])
            count[actno][1]+=int(dataset.loc[i,"Adult Victims"])

In [ ]:
children_affected=[]
adults_affected=[]
for i in range(1,12):
    children_affected.append(count[i][0])
    adults_affected.append(count[i][1])
index=np.arange(11)
width=0.3
plt.figure(figsize=(9,9))
plt.bar(index,children_affected,width,color='red',label = 'Children affected')
plt.bar(index+width,adults_affected,width,color='blue',label = 'Adults affected')
plt.xticks(index+width/2,act_key,rotation=90,fontsize=13)
plt.legend(loc='best',prop={'size':20})
plt.xlabel("Acts",fontsize=16)
plt.ylabel("# of people affected",fontsize=16)
plt.title("Number of people affected by different acts",fontsize=20)
plt.show()